In [ ]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=8582e169b703b8b10409ce54b9ca56bd8ade0561c23f92417471e7dcec587d1b
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


In [ ]:
import pandas as pd
from transformers import pipeline
!pip install pdfplumber
import pdfplumber
import re
from googletrans import Translator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import pdfplumber
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from googletrans import Translator

translator = Translator()

def extract_clean_text_chunks_from_pdf(pdf_path, chunk_size=1000):
    text_chunks = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                page_text = re.sub(r'http\S+|www\S+|file:\S+|\S+\.html', '', page_text)
                page_text = re.sub(r'\s+', ' ', page_text).strip()
                for i in range(0, len(page_text), chunk_size):
                    chunk = page_text[i:i + chunk_size]
                    if chunk:
                        text_chunks.append(chunk)
    return text_chunks

question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qa-qg-hl")

def generate_questions(text, num_questions=5):
    if not text:
        return []
    formatted_text = "generate question: " + text
    questions = question_generator(formatted_text, max_length=100, num_beams=5, num_return_sequences=num_questions)
    return questions

def retrieve_relevant_chunks(prompt, text_chunks, top_n=5):
    if not text_chunks:
        return []
    vectorizer = TfidfVectorizer().fit_transform([prompt] + text_chunks)
    cosine_similarities = cosine_similarity(vectorizer[0:1], vectorizer[1:]).flatten()
    top_n_indices = cosine_similarities.argsort()[-top_n:][::-1]
    relevant_chunks = [text_chunks[i] for i in top_n_indices]
    return relevant_chunks

def safe_translate(text, src, dest):
    try:
        if text is None or text.strip() == "":
            return ""
        translated_text = translator.translate(text, src=src, dest=dest).text
        return translated_text
    except Exception as e:
        return ""

def generate_questions_from_prompt_with_rag(prompt, pdf_text, total_questions=10, top_n_chunks=5, questions_per_chunk=3):
    translated_prompt = safe_translate(prompt, src='hi', dest='en')

    if not translated_prompt:
        return []

    relevant_chunks = retrieve_relevant_chunks(translated_prompt, pdf_text, top_n=top_n_chunks)

    if not relevant_chunks:
        return []

    all_generated_questions = []
    for chunk in relevant_chunks:
        translated_chunk = safe_translate(chunk, src='hi', dest='en')
        if not translated_chunk:
            continue

        questions = generate_questions(translated_chunk, num_questions=questions_per_chunk)

        for q in questions:
            cleaned_question = re.sub(r'\s+', ' ', q['generated_text']).strip()
            if cleaned_question:
                all_generated_questions.append(cleaned_question)

        if len(all_generated_questions) >= total_questions:
            break

    translated_questions = []
    for q in all_generated_questions:
        translated_q = safe_translate(q, src='en', dest='hi')
        if translated_q:
            translated_questions.append(translated_q)

    return translated_questions


prompt = input("Enter a prompt (in Sanskrit): ")
pdf_path = "/content/narayan-kwach.pdf"

text_chunks = extract_clean_text_chunks_from_pdf(pdf_path)

if text_chunks:
    print(f"Extracted {len(text_chunks)} text chunks.")
else:
    print("No text found in the PDF.")

if text_chunks:
    print(f"Generating questions based on the prompt: '{prompt}'...")
    generated_questions = generate_questions_from_prompt_with_rag(
       prompt, text_chunks, total_questions=5, top_n_chunks=5, questions_per_chunk=2
    )

    if generated_questions:
        print("Generated Questions:")
        for idx, question in enumerate(generated_questions, 1):
           print(f"{idx}. {question}")
    else:
        print("No questions were generated.")
